# Linear Regression 

# Manually Computing the Gradient 

In [7]:
import numpy as np
from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [28]:
ones = np.ones((m, 1))

hdpb = np.c_[ones, housing.data]

ones.shape, housing.data.shape, hdpb.shape

((20640, 1), (20640, 8), (20640, 9))

In [29]:
theta_value

array([[-3.6412704e+01],
       [ 4.3792546e-01],
       [ 9.5298719e-03],
       [-1.0845181e-01],
       [ 6.4850527e-01],
       [-3.6632082e-06],
       [-3.8002990e-03],
       [-4.1567561e-01],
       [-4.2836216e-01]], dtype=float32)

In [22]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy, housing.target.shape[-1])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]] 20640


In [33]:
n_epochs = 1000
learning_rate = 0.01

tf.reset_default_graph()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()


Epoch 0 MSE = 2.7544265
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727804
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293705


In [34]:
best_theta

array([[ 2.0685523e+00],
       [ 7.7407807e-01],
       [ 1.3119240e-01],
       [-1.1784509e-01],
       [ 1.6477816e-01],
       [ 7.4408489e-04],
       [-3.9194517e-02],
       [-8.6135650e-01],
       [-8.2347965e-01]], dtype=float32)

# Using autodiff

In [32]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [35]:
gradients = tf.gradients(mse, [theta])[0]

In [36]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.7544265
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727804
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.537379
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124243
Epoch 900 MSE = 0.5293705
Best theta:
[[ 2.06855249e+00]
 [ 7.74078012e-01]
 [ 1.31192341e-01]
 [-1.17845066e-01]
 [ 1.64778158e-01]
 [ 7.44066201e-04]
 [-3.91945057e-02]
 [-8.61356854e-01]
 [-8.23479950e-01]]


# autodiff - Partial Differential Equation 

In [37]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [38]:
my_func(0.2, 0.3)

-0.21253923284754914

WE NEED to Calculate Gradients

In [39]:
tf.reset_default_graph()

a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

In [40]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.21253741
[-1.1388494, 0.19671397]


# Using a GradientDescentOptimizer

In [41]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [42]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [43]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.7544265
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727804
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.537379
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124243
Epoch 900 MSE = 0.5293705
Best theta:
[[ 2.06855249e+00]
 [ 7.74078012e-01]
 [ 1.31192341e-01]
 [-1.17845066e-01]
 [ 1.64778158e-01]
 [ 7.44066201e-04]
 [-3.91945057e-02]
 [-8.61356854e-01]
 [-8.23479950e-01]]


# Using a momentum optimizer

In [44]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [45]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

In [46]:
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [47]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Best theta:
[[ 2.068558  ]
 [ 0.82961684]
 [ 0.11875118]
 [-0.26552188]
 [ 0.30569202]
 [-0.00450316]
 [-0.03932616]
 [-0.8998903 ]
 [-0.87054545]]


# Mini-batch Gradient Descent

In [60]:
n_epochs = 10
learning_rate = 0.01

In [61]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [62]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

In [63]:
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [64]:
init = tf.global_variables_initializer()

In [65]:
batch_size = 100

n_batches = int(np.ceil(m / batch_size))

In [66]:
n_batches

207

In [85]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    #print(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)  
    #print(indices)
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    #print(X_batch.shape, y_batch.shape)
    return X_batch, y_batch

In [69]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

0
[ 2732 10799  9845 19648 13123  9225 14116 14935 15430 15832  6744 19852
 14650 17089 18983 10327 18606  2897 16921  6216  6921  6036  2163  5072
  4851  7877 18430  1871  7599  2496 15186 10200   755   797 17043  3219
 15246 16321 19721 19129  8736 17098 18676 11723 11122 17591 10368  6021
 20006  3560 12676 18025 13754 14879 14627 12134 15115  8622 20571 19043
  9781 19340 16298 12372 18635  4420 13062  7108 17455 14324  5251  7012
  9396 14312  3918  9359 18068 11491 15127 10959  2957  4469 14165  8752
  9797 14505  5795 17856  7365  8448  3762 16221 18819  1634 15914   973
 10215 11157 13824  2418]
(100, 9) (100, 1)
1
[  235 12172  5192 17289 10955  7813 19279   144 20609 16332  7751 10989
  3462 12645  2962 18900  8444 19946  4764 16946 19433 17137 11742 18272
  1110  6285  1031  4415 19261  3606  9529  2561  6771   431  4074 14792
 16926  9287 13699  1478 16277  1337 12803 11032  8689  1612 19482  1844
  9552  2669    15 18628 12313 11631 13783 10119  1306  9832  1046 14089
 16

(100, 9) (100, 1)
105
[12917 11030    73 15588  6944 19136 13488 10241  3375  9417  8861 10014
  5395  3100   543 13081  1374   989 11164   925  6678 13593 10260 11018
  4029 12895 11281  1804 14906  6195 13977 17341 13385  2886  7543   768
  7314 19694  3766  5117 19774 13770   508  2008  9566  7641  6739 11060
 11371  3096 18160 12866  8192   618  2809 11561  5795  6544 18574  7139
 16178 12945  9680  8255 18047 15337 17378  3706 15308 10670 16284 19211
   809  2648  3592  6300 11083 10580 17982 14300  3837   283  1941  5173
  7270 18899 12348 18537 19783  8339 13328  1027  1478 17715 17223 13656
 10857 15195  4565 10041]
(100, 9) (100, 1)
106
[19822  9178 18329 12735  2580 19508 14538  6035 10869  6914 16109  7957
 11948 15205 14086  8761 13284     5 10877 10072 16402  4699 18635 10399
  5866 16200 12773  4724 19184 19024   441 16886 19144 12960  4925  7826
  9599 11662 17321 12117 13850  6619 20241 13462 11857  8624 11917  5962
  6362   142  6589  8582  4283  9193 17599  1277  7284

(100, 9) (100, 1)
201
[17961  7680   501  1891  9082 10428  7903  1295 10403 17946  2381 20504
  8034 14624  9187 20329 17957 13712  6277 15354  7816 19298  8884  1563
 10328 11799 13222 10896  5682 19476 18152  3504 13813  8184   232  4049
 11492  3580  1910 15445   603 15542 18960  5851   974 13143  9562  1520
 16266 13434  1244  7360 14652 15165 11132  8860  7576 14926  8618  9644
  7132 12036 12150  8284  2301  2402   745   725 10402  9953 19712  5248
  2887 10654  2775  4170  7907 19211  4282 16642  6385 18744 18952 11720
 11926  3263  1882  9374 20474 11289  8180  5748  2273  5271 15605 15351
  9108  3865  7307  5577]
(100, 9) (100, 1)
202
[ 5011 10903  4156  4009  9401 19814 14465 14279  8924 10387 14089 12413
  1972 18775 19573  7784 12169  3115 14455 17731  5104  8579  2737 20598
 19898 11900 13680  5769 10307 19850  6114 10361 20215 12118 15397  4355
  6802 12714 19246  3792  1210  7669   850  4438 15116 15127  9581  9178
 11315  2671 12085 14906 11549 16447  2413   693 10780

(100, 9) (100, 1)
287
[16683 11723  2830  1338 10334 14850 11303  4301  9467 19490  6231  9337
 14741 18548 20498 13337  9844 14891  6186 12683   906 16199 14029  4324
  8380  8485 13324 13996  8914  4841   928 15366  3460 11658 10512  3749
  1965 15759 17226 15548 18611  6361  2771  3458 18339 14513 16246 14393
 10460  9724 14698  3121  8254 11525 16623   149  8783 15102 18935  5399
 18167  3997 14275 16171 13972 13247  3467  3302  8788 10642 11863  2953
 10112   613 14405 18453  8226 18443   484  1054   672 13412 13014 12557
  8432 15230 14056 18367  6650 15293  9822 16007  3243 17370  3437  2226
 17509  3632  7558 19607]
(100, 9) (100, 1)
288
[ 5282  5961  3544  8099 11916  8895  4234 15291  9282 12651 13345 18531
 11827 15557 14652 16632 19313 14601 19439 15060  7714  7418  2765 11544
   739  4938  2965 18566  4180  4501  2589  9982  6996 15479   279  8296
  7083  4187 12225 10765 19986  1985  4626 10835  6165  3397 14959  1365
  5699  6435  2162  4394 16819 20439  9296  7648  3639

(100, 9) (100, 1)
377
[12709  6154  9387  5741 14985  8491 18151  4008 18042  8097 12775 15173
 14332 16639 13959   285 15549  6256 19060 20534  6146 10873 13633  7323
 17367 11858 15775  2701  3668 14814 17056  2492 19936 19964 13246 10168
 17842 15538 17198  7429 20121  8173  3849 17787 10275  6527  1165   313
  7516 18820 19231  2052  1873  1112 16408  9422  6453  4368  8127  3116
 12439 18138 19599 18264  5055 11298  9119 14001 14270 12445 15772  8165
 14192 11168 13042  1420 16317 20601 10983 11930  5451 16348  7675 15444
  5236 18658 12030 14158 15894   901  4843  5397  7359 12155 18536 14795
   897 10400 15845 13397]
(100, 9) (100, 1)
378
[ 8436  1989  5158  7719  3894  4284 13393 19031 14032 15601  8812 10136
 14875 13364  6111 11398  2892 12807  5010 14250 18855  6812 13568  4605
  3213  8365 13141  9221  1506 11479 17964  6072  6567  9291  7726 13669
 10537 14690  6589  7259 18783 17405 10693 16318 15125  3794  9524  4245
  3376   343 12548  7990 18669  6207 10883  1075 17403

...

1689
[18651 17723  4080 16831 13329 12869 11457 12528 10156  1171 18340 12907
  8752  8191 10387 18865 20138   555  9881 15374 11606  5608 10938   318
  8890 12741  4016 17009  7797  4196 13357  7484  1916 16136  3725 14713
  4422  4829  3062  6092   523 15451 18160 10359  8633 11890  9084  6824
 16863 18570  9985  9363  7777  1430  6897  6963  5828 12893 15905  7069
 10557  9840  1949  1793 14795 14067 20502  9686  4687 10345  1775 10191
  4962  8436 16048  7349  7040  7675 19155 11251 15535 14644  5188 14919
 16210  8560 14173 17885 20226 18739   666  2421 15986  2162 20627 19844
 16841  6262 20195 14607]
(100, 9) (100, 1)
1690
[  254 20341 11358 20594 10024 15916 18229 14929 20018  4894 12218 14476
  3407 11707  4442 20412  8924  2883 11091 14508  4400  8251 17838  3573
 12438  1054 17930  9055 15820  2257 13695  2964  3395 12205  2582 12929
  9059  7164 10854   845 13159 11374 20043   516  4212 14054  8789 10990
  3286   669 11535 20207 19996 19904  7307 17252  8503 14372   443 128

(100, 9) (100, 1)
1782
[11156 16167 12952    52 19403  7368  4414  6929  3717 20167  9879 19279
  6496  1498 15639 12567 15995 13714  5436 12935  4596 13108 17844  9277
 19373  4293 10476 16632 12536  2213 18263  3031 20105  1508 10946 19409
   158  3507  8726 13091  6277  7297  4991 16600 20472  7196 12067   657
   887 11999  6050   697  1790 14934  4667 17933 12466 17858 20370  6539
  7516 14837 12491  8230 10138  8016  5334 13288  7629  8083 10337 15428
 13532 14991 11918   103 11671 15168  2164  5084 16179  7898 17959 16076
 14470 15864  8255 19180  1576  6884  4676 11260 12890 11530  3381 13946
 12794 11612 20334 17851]
(100, 9) (100, 1)
1783
[15866 13689 17124  3617 18301   939  5625 11012 12623  5888  1532 14297
  8802  7409  5962 13483  2117 16699 12531 11569  3473  4847 16245  4193
  3364 16163 19697  8204    27 15586  1814 13437 15647 20204  1529 19636
   148  1561 13203 19208 14984 12980 12582  1078 10203 11020 11545   158
 17638 13067  5092  7016  8042  4503 10760 19120 173

[ 1344 19419 12226  1696  6314 16408 18980 13007 10970  2111   700  5964
 20258  6590  9507 12866  8357  6416  3175  5781 16481 15740 14733  4270
  6581  8298  5540  5785  2171    27 15151  6080 18491 20321  7936 18905
 11005  5304  2732 17601  9056  1735 17730 18818   307 13649 14507  4978
 17918 10013  5041   394 11792   236 13795  7360  6554 16559 16772 13972
 17543  1690 13797 16085  6183 14368  2293 10580  1601  1143  2684 11373
  2000  5188 18772  7809  7931  8626  1651  2525 10397  3597  8255  7772
 17370 17614 16838  6955 17370  7202 17881 12546  1078   120  4212 12373
 11211 11331  9415  3529]
(100, 9) (100, 1)
1861
[10871 10548  6675  8432 13479 19508 14818 16943 13924 20385  4689  5456
  4219  5234  8907  5023 10358 20256  2408 12042  4183 10475  6551 20221
 17109 19536 16811 10213 10895 13518 18990  7873 12420  2525  8588  7175
  2769   677  7673  3223 20076 10188 19095  9634  8041   988 16818  2517
  3096  6823  7758   670  3943 12310 16530 12319 15835  5990 18906  5610
 2

1947
[20552 12590 17626 18888 15725 18140 19293  7339 12665  8403 17977 10835
 15148  6783 16992 17877  9843  5849 10173   998 19184 12212  5434 10548
 11820 13284  9831 18683  9288 13204 16238 11816  1083  7570 18631  9038
   916 12346 14174 15339 18446 15197 13055  7730 11481  7726  3199  6190
 13268  6148 20194 14076 10686 17685 17992 16691  5121  1394  8117  8306
 10205  8832 19258  4541 16410 11819 12535 16365 18528 18658 11655  1604
  5321  6161 10519 19128 12971   347 10181 17395 12539 16097  5171  6816
  4955 15500 11913 11335 15787 19308 18269 13323 16159 20467  5994 13039
 10379  6341 20387 10475]
(100, 9) (100, 1)
1948
[12587 13486  9093  5448 15150 12294  6514  4571 20099 18521 12865 10516
 11405 14804 18691 15922  6407  8337 13594  6404  8660 13955  5061 18434
 12068 12691  3889 18719 10649  6912  5345 13858  2205  1641  6100 16512
  8266  5230  2507 12957  9708  5355 14507 19198 19260 17701 12578 14484
 17283  8218 17002  6073 12488 18579  3217 16766 13683  7834 12056 202

(100, 9) (100, 1)
2068
[ 7582 18678  7412   876 12472 10711  3619  6314   231 18643 19683  7532
  1819  1986 17355  6789  4477  2458  4551  8814 14459  5150  5801 11788
  9009 15534 15603 12072  2641  9802 16141 14943  9523 19310 18622  8783
  2241 12647  5827  5992 13031 16802 13405 19967 15230  3454  1601  1768
 20612  8281 17896 17217  3763  1948 16316  9272 18491 19972  7030  7315
 17232  5310   187 20529 10264 13774  5195  7042  8549 20284 17593  8435
  6824  5674  4372 14015  4934 15274  8330   451  8790  1949 18203 11565
 17670  1963 10713  3098 10881 10259 11951  6666  4594 14175 19566  7107
  6059  4820  1039 18222]
(100, 9) (100, 1)
2069
[16349 18108 15657  8938 17461   799  6596  3613  4912 12263 16667  5556
  6726  4414 11373 16508  8053  1916  8620  5095  4551 16536  5940 15465
  9073 19286   876   906 14313  5674 15080 10780  7026 15584 17978 20385
 14536 13933 19487  1696 20020 12927  8985  9461 10993 11194 18800  1597
 10552 20632 15446  4016  7553 10084  3109  4862  61

In [70]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.11731728],
       [-0.22739054],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.9164394 ],
       [-0.87950087]], dtype=float32)

# Saving and restoring a model

In [72]:
tf.reset_default_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "./model_file/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./model_file/my_model_final.ckpt")

Epoch 0 MSE = 2.7544265
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727804
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.537379
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124243
Epoch 900 MSE = 0.5293705


In [73]:
best_theta

array([[ 2.06855249e+00],
       [ 7.74078012e-01],
       [ 1.31192341e-01],
       [-1.17845066e-01],
       [ 1.64778158e-01],
       [ 7.44066201e-04],
       [-3.91945057e-02],
       [-8.61356854e-01],
       [-8.23479950e-01]], dtype=float32)

If you want to have a saver that loads and restores theta
with a different name, such as "weights":

saver = tf.train.Saver({"weights": theta})

In [74]:
with tf.Session() as sess:
    saver.restore(sess, "./model_file/my_model_final.ckpt")
    best_theta_restored = theta.eval()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model_file/my_model_final.ckpt


In [78]:
# notice that we start with an empty graph.
tf.reset_default_graph()

saver = tf.train.import_meta_graph("./model_file/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "./model_file/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from ./model_file/my_model_final.ckpt


In [ ]:
import os, argparse

dir = os.path.dirname(os.path.realpath(__file__))


In [79]:
best_theta_restored

array([[ 2.06855249e+00],
       [ 7.74078012e-01],
       [ 1.31192341e-01],
       [-1.17845066e-01],
       [ 1.64778158e-01],
       [ 7.44066201e-04],
       [-3.91945057e-02],
       [-8.61356854e-01],
       [-8.23479950e-01]], dtype=float32)

In [77]:
np.allclose(best_theta, best_theta_restored)

True

# Using TensorBoard

In [86]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [87]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [88]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [89]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [90]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [91]:
file_writer.close()

In [92]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.11731728],
       [-0.22739054],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.9164394 ],
       [-0.87950087]], dtype=float32)